# Intermediate-Statistics-Exercise
## Regression: MPG
###6031010021 Yanika Dontong

In [128]:
#Importing required libraries
#Importing the required libraries and data set 
import numpy as np
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
%matplotlib inline

print("Important libraries loaded successfully")

Important libraries loaded successfully


Data Dict: 
![](https://i.ibb.co/N9SWSmk/Screenshot-2563-02-07-at-22-06-32.png)
![](https://i.ibb.co/TTWZ09B/Screenshot-2563-02-07-at-22-06-41.png)

### Q1 Mpg (con) ~ horsepower (continous) – Correlation
### Q2 Mpg (con) ~ model (category) – F-test, ANOVA
### Q3 Mpg (con) ~ all continuous variables & report R^2 & RMSE – Regression
### Q4 Mpg (con) ~ use SFS to select 3 continuous variables & report R^2 & RMSE – Regression

In [129]:
mpg_df = pd.read_csv('https://github.com/kaopanboonyuen/Python-Data-Science/raw/master/Dataset/auto-mpg.csv')

In [130]:
mpg_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [131]:
mpg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


In [132]:
# Write your code by following 4 questions
import scipy.stats
# Q1 Mpg (con) ~ horsepower (continous) – Correlation
def drop_non_numeric(series):
  if series.isnumeric():
    return float(series)
  else:
    return 0.
mpg_df.dropna(inplace=True)
mpg_df.corr(method = 'pearson')
mpg_df = mpg_df[mpg_df.horsepower != '?']
x = mpg_df['mpg']
y = mpg_df["horsepower"].apply(drop_non_numeric)
r,p = scipy.stats.pearsonr(x, y)
print(r,p)

-0.7784267838977761 7.031989029404151e-81


In [133]:
# Q2 Mpg (con) ~ model (category) – F-test, ANOVA
model  = mpg_df['model year']
data2 = pd.DataFrame({"mpg":mpg_df['mpg'],"model":model})
# print(data2.groupby('model').mean())
lm = ols('mpg ~ C(model)',data=data2).fit()
table = sm.stats.anova_lm(lm)
print(table)

             df        sum_sq     mean_sq          F        PR(>F)
C(model)   12.0  10236.296801  853.024733  23.802076  1.986801e-39
Residual  379.0  13582.696668   35.838250        NaN           NaN


In [134]:
overall_mean = data2['mpg'].mean()
data2['overall_mean'] = overall_mean
#SST
ss_total = sum((data2['mpg']-data2['overall_mean'])**2)
print("SST: ",ss_total)
group_means = data2.groupby('model').mean()
group_means = group_means.rename(columns = {'mpg': 'group_mean'})
# print(group_means)
data2 = data2.merge(group_means, left_on = 'model', right_index = True)
# print(data2)
# SSE
ss_residual = sum((data2['mpg'] - data2['group_mean'])**2)
print("SSE: ",ss_residual)
#SSM
overall_mean = data2['mpg'].mean()
ss_explained = sum((overall_mean - data2['group_mean'])**2)
print("SSM: ",ss_explained)
#MSE
n_groups = len(set(data2['model']))
n_obs = data2.shape[0]
df_residual = n_obs - n_groups
ms_residual = ss_residual / df_residual
print("MSE: ",ms_residual)
#MSM
df_explained = n_groups - 1
ms_explained = ss_explained / df_explained
print("MSM: ",ms_explained)
#F-test
f = ms_explained / ms_residual
print("F-test: ",f)

SST:  23818.993469387755
SSE:  13582.69666827444
SSM:  10236.296801113307
MSE:  35.83824978436528
MSM:  853.0247334261089
F-test:  23.80207567497472


In [135]:
# Q3 Mpg (con) ~ all continuous variables & report R^2 & RMSE – Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

data3 = mpg_df.drop(['horsepower','car name'],axis=1)
# print(data3.info())
X = data3.drop("mpg",axis=1)
Y = data3['mpg']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2020)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
print('R2_score: %.2f'
      % r2_score(y_test, y_pred))


Mean squared error: 13.33
R2_score: 0.76


In [136]:
# Q4 Mpg (con) ~ use SFS to select 3 continuous variables & report R^2 & RMSE
import statsmodels.api as sm
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import utils
data4 = mpg_df.drop(['horsepower','car name'],axis=1)

X_opt = X
regressor_OLS = sm.OLS(endog =y, exog = X_opt).fit()
# print(regressor_OLS.summary())
clf = LogisticRegression()
sfs1 = sfs(clf,
           k_features=3,
           forward=True,
           floating=False,
           verbose=2,
           scoring='r2',
           cv=5)

lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
sfs1.fit(X,training_scores_encoded)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED

SequentialFeatureSelector(clone_estimator=True, cv=5,
                          estimator=LogisticRegression(C=1.0, class_weight=None,
                                                       dual=False,
                                                       fit_intercept=True,
                                                       intercept_scaling=1,
                                                       l1_ratio=None,
                                                       max_iter=100,
                                                       multi_class='auto',
                                                       n_jobs=None,
                                                       penalty='l2',
                                                       random_state=None,
                                                       solver='lbfgs',
                                                       tol=0.0001, verbose=0,
                                                       warm_start=False),

In [137]:
feat_cols = list(sfs1.k_feature_names_)
print("Selected 3 variables: ",feat_cols)

Selected 3 variables:  ['cylinders', 'displacement', 'model year']


In [147]:
X = pd.DataFrame({'cylinders':data4['cylinders'],'displacement':data4['displacement'],'model year':data4['model year']})
Y = data4['mpg']
# print(Y.head())
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2020)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
print('R2_score: %.2f'
      % r2_score(y_test, y_pred))

Mean squared error: 18.36
R2_score: 0.68
